In [1]:
import pandas as pd
import spacy
import torch

from collections import defaultdict
from itertools import chain

from sklearn.model_selection import train_test_split
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torchtext import data

from quoraquestionpairs.data import get_dataset
from quoraquestionpairs.neuralnets import RNNGRUSequential
from quoraquestionpairs.training import train

In [2]:
import importlib

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
nlp = spacy.load('en')

In [5]:
question = data.Field(tokenize='spacy',
                      lower=True,
                      pad_first=True)

In [6]:
data_df = get_dataset('data/train.csv', question)

In [7]:
train_ds, val_ds = data_df.split(0.95, stratified=True, strata_field='is_duplicate')

In [8]:
question.build_vocab(train_ds,
                     min_freq=50)

### Neural net structure
Input: two sequences
[?1 x 1], [?2 x 1]

Embedding layer:
-> [n_vocab, embedding_dim] -> [?1 x embedding_dim], [?2 x embedding_dim]

Concatenation:
-> [?1 + ?2 x embedding_dim]

GRU:
-> [embedding_dim

In [9]:
# x1 = torch.tensor([1, 2, 0])
# x2 = torch.tensor([2, 0])

# emb = nn.Embedding(3, 10)

# x1_emb = emb(x1)
# x2_emb = emb(x2)

# assert x1_emb.size() == torch.Size([3, 10]) and x2_emb.size() == torch.Size([2, 10])

# concatenated = torch.cat([x1_emb, x2_emb])

# assert concatenated.size() == torch.Size([5, 10])

# gru = nn.GRU(input_size=10, hidden_size=8)
# _, x = gru(concatenated.view(-1, 1, 10))

# assert x.size() == torch.Size([1, 1, 8])

# linear = nn.Linear(8, 1)

# out = torch.sigmoid(linear(x))

In [10]:
rnn = RNNGRUSequential(len(question.vocab), 300, 128).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.01)

In [11]:
train_iter = data.Iterator(train_ds,
                           batch_size=256,
                           repeat=False,
                           shuffle=True,
                           device=device)
    
val_iter = data.Iterator(val_ds,
                         batch_size=256,
                         device=device)

In [13]:
train(rnn, criterion, optimizer, 10, train_iter, val_iter, device)

Samples processed: 100096 Loss: 0.002081677955670563
Samples processed: 100096 Loss: 0.0019575727098952514
Samples processed: 100096 Loss: 0.0019197972640847725
Epoch: 1 Val accuracy: 76.10759493670885
Samples processed: 100096 Loss: 0.0018354402074966666
Samples processed: 100096 Loss: 0.0018658884652101857
Samples processed: 100096 Loss: 0.0018575213583962768


KeyboardInterrupt: 

In [ ]:
for epoch in range(10):
    
    running_loss = 0.0
    for i, ((x1, x2), target) in enumerate(train_iter):
        x1 = x1.to(device)
        x2 = x2.to(device)
        target = target.type(torch.FloatTensor).to(device) #target.type(torch.FloatTensor).to(device)
        
        optimizer.zero_grad()
        
        output = rnn(x1, x2)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i % 250 == 0:
            print('Batch: {} Loss: {}'.format(i + 1, running_loss / 250))
            running_loss = 0.0
    
    correct = 0
    total = 0
    with torch.no_grad():
        for (x1, x2), target in val_iter:
            x1 = x1.to(device)
            x2 = x2.to(device)
            target = target.type(torch.ByteTensor).to(device)
            output = rnn(x1, x2)
            output = output
            pred = output >= 0.5
            correct += (target == pred).sum().item()
            total += 256
        
    print('Epoch: {} Val accuracy: {}'.format(epoch + 1, correct / total * 100))

In [ ]:
class QuoraDataset(Dataset):
    
    def __init__(self, csv_file, train=False, transform=None):
        self.questions = pd.read_csv(csv_file, keep_default_na=False)
        self.train = train
        self.transform = transform
    
    def __len__(self):
        return len(self.questions)
    
    def __getitem__(self, idx):
        
        sample = {'q1': self.questions.loc[idx, 'question1'],
                  'q2': self.questions.loc[idx, 'question2']}
        
        if self.train:
            sample['target'] = self.questions.loc[idx, 'is_duplicate']
            
        if self.transform:
            sample = self.transform(sample)
        
        return sample
    
    def get_tokens(self):
        

In [ ]:
dataset = QuoraDataset('train.csv', train=True)

In [ ]:
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4)

In [ ]:
tokenizer = Tokenizer(nlp.vocab)

In [ ]:
word_dict = defaultdict(int)

for i in range(len(dataset)):
    sample = dataset[i]
    questions = [tokenizer(sample['q1']), tokenizer(sample['q2'])]
    for token in chain(*questions):
        word = token.text.lower()
        if token.is_stop:
            continue
        else:
            word_dict[word] += 1

In [ ]:
sorted_words = sorted([(word, counts) for word, counts in word_dict.items()], key=lambda x: x[1], reverse=True)

In [ ]:
class WordTokenizer:
    
    def __init__(self, num_words):
        self.num_words = num_words
    
    def tokenize(self, *args):
        words_counts = defaultdict(int)
        
        for text in chain(*args):
            for token in tokenizer(text):
                word = token.text.lower()
                if token.is_stop:
                    continue
                else:
                    words_counts[word] += 1
        
        words_counts = sorted([(word, counts) for word, counts in words_counts.items()], key=lambda x: x[1], reverse=True)
        self.words_dict = {word: i for i, (word, counts) in enumerate(sorted_words, 1)}
    
    def __call__(self, seq):
        int_seq = [self.words_dict[word] for word in ]

In [ ]:
#{word: i for i, (word, counts) in enumerate(sorted_words, 1)}

In [ ]:
def test_fun(*args):
    for i in chain(*args):
        print(i)

In [ ]:
for i in dataset.questions['question1'].values:
    print(i)
    break

In [ ]:
word_tokenizer = WordTokenizer(100)

In [ ]:
#test_fun(dataset.questions['question1'].values, dataset.questions['question2'].values)

In [ ]:
word_tokenizer.tokenize(dataset.questions['question1'].values, dataset.questions['question2'].values)